In [1]:
# Import statements

import os
import numpy as np
from scipy.spatial import distance_matrix
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import re
from scipy.ndimage import distance_transform_edt
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.spatial.distance import cdist
import pandas as pd

In [2]:
# Functions

# Ensure coordinates are in the correct shape (2D array of shape (n_points, 3))
def load_swc_coordinates(file_path):
    # Assuming this function loads the SWC data and returns it as a numpy array (n_points, 3)
    # The last column might be radius, so we take only the first three columns (x, y, z)
    data = np.loadtxt(file_path)
    return data[:, 2:5]  # Return only the x, y, z columns

def average_bidirectional_distance(A, B):
    # Ensure that A and B are 2D arrays
    A = np.array(A)
    B = np.array(B)
    
    # Check if the coordinates are 2D arrays
    if A.ndim != 2 or B.ndim != 2:
        raise ValueError("Both A and B should be 2D arrays representing the coordinates (n_points, 3).")

    # Compute pairwise distance matrix between A and B
    dists = cdist(A, B)
    
    # Average distances A->B and B->A
    avg_a_to_b = np.mean(np.min(dists, axis=1))  # Minimum distance for each point in A to closest point in B
    avg_b_to_a = np.mean(np.min(dists, axis=0))  # Minimum distance for each point in B to closest point in A
    
    return (avg_a_to_b + avg_b_to_a) / 2

def compute_pairwise_distances(swc_folder):
    swc_files = glob.glob(os.path.join(swc_folder, '*.swc'))
    pairwise_means = []
    
    # Assuming you have a function to compute pairwise distances (e.g., `average_bidirectional_distance`)
    for i, file_i in enumerate(swc_files):
        coords_i = load_swc_coordinates(file_i)
        for j, file_j in enumerate(swc_files[i + 1:], start=i + 1):
            coords_j = load_swc_coordinates(file_j)
            pairwise_means.append(average_bidirectional_distance(coords_i, coords_j))
    
    return pairwise_means


In [3]:
## Data wrangling for GABA skeletons

# Load and compute pairwise distances for GABA left and right datasets affine+rigid and diffeomorphic
left_GABA_affine_folder = 'skeletons_affine+rigid/GABA/left/'
right_GABA_affine_folder = 'skeletons_affine+rigid/GABA/right/'

left_GABA_affine_pairwise_means = compute_pairwise_distances(left_GABA_affine_folder)
right_GABA_affine_pairwise_means = compute_pairwise_distances(right_GABA_affine_folder)

left_GABA_diffeo_folder = 'skeletons_diffeomorphic/GABA/left/'
right_GABA_diffeo_folder = 'skeletons_diffeomorphic/GABA/right/'

left_GABA_diffeo_pairwise_means = compute_pairwise_distances(left_GABA_diffeo_folder)
right_GABA_diffeo_pairwise_means = compute_pairwise_distances(right_GABA_diffeo_folder)

# Calculate means and standard deviations
mean_GABA_affine_left = np.mean(left_GABA_affine_pairwise_means)
std_GABA_affine_left = np.std(left_GABA_affine_pairwise_means)

mean_GABA_affine_right = np.mean(right_GABA_affine_pairwise_means)
std_GABA_affine_right = np.std(right_GABA_affine_pairwise_means)

mean_GABA_diffeo_left = np.mean(left_GABA_diffeo_pairwise_means)
std_GABA_diffeo_left = np.std(left_GABA_diffeo_pairwise_means)

mean_GABA_diffeo_right = np.mean(right_GABA_diffeo_pairwise_means)
std_GABA_diffeo_right = np.std(right_GABA_diffeo_pairwise_means)

# Create a DataFrame to combine both sets of data
df_GABA_affine_left = pd.DataFrame({
    "pairwise_mean": left_GABA_affine_pairwise_means,
    "set": "Left"
})

df_GABA_affine_right = pd.DataFrame({
    "pairwise_mean": right_GABA_affine_pairwise_means,
    "set": "Right"
})

df_GABA_affine = pd.concat([df_GABA_affine_left, df_GABA_affine_right], ignore_index=True)

df_GABA_diffeo_left = pd.DataFrame({
    "pairwise_mean": left_GABA_diffeo_pairwise_means,
    "set": "Left"
})

df_GABA_diffeo_right = pd.DataFrame({
    "pairwise_mean": right_GABA_diffeo_pairwise_means,
    "set": "Right"
})

df_GABA_diffeo = pd.concat([df_GABA_diffeo_left, df_GABA_diffeo_right], ignore_index=True)


/Users/lindsey/Documents/GitHub/FrankandLopes2025-RefBrain/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lindsey/Documents/GitHub/FrankandLopes2025-RefBrain/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/lindsey/Documents/GitHub/FrankandLopes2025-RefBrain/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/lindsey/Documents/GitHub/FrankandLopes2025-RefBrain/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/lindsey/Documents/GitHub/FrankandLopes2025-RefBrain/.venv/lib/python3.9/site-packages/numpy/core/_methods

In [4]:
# Data wrangling for inotocin skeletons 

# Load and compute pairwise distances for inotocin left and right datasets (affine+rigid and diffeomorphic)
left_inotocin_affine_folder = 'skeletons_affine+rigid/inotocin/left/'
right_inotocin_affine_folder = 'skeletons_affine+rigid/inotocin/right/'

left_inotocin_affine_pairwise_means = compute_pairwise_distances(left_inotocin_affine_folder)
right_inotocin_affine_pairwise_means = compute_pairwise_distances(right_inotocin_affine_folder)

left_inotocin_diffeo_folder = 'skeletons_diffeomorphic/inotocin/left/'
right_inotocin_diffeo_folder = 'skeletons_diffeomorphic/inotocin/right/'

left_inotocin_diffeo_pairwise_means = compute_pairwise_distances(left_inotocin_diffeo_folder)
right_inotocin_diffeo_pairwise_means = compute_pairwise_distances(right_inotocin_diffeo_folder)

# Calculate means and standard deviations
mean_inotocin_affine_left = np.mean(left_inotocin_affine_pairwise_means)
std_inotocin_affine_left = np.std(left_inotocin_affine_pairwise_means)

mean_inotocin_affine_right = np.mean(right_inotocin_affine_pairwise_means)
std_inotocin_affine_right = np.std(right_inotocin_affine_pairwise_means)

mean_inotocin_diffeo_left = np.mean(left_inotocin_diffeo_pairwise_means)
std_inotocin_diffeo_left = np.std(left_inotocin_diffeo_pairwise_means)

mean_inotocin_diffeo_right = np.mean(right_inotocin_diffeo_pairwise_means)
std_inotocin_diffeo_right = np.std(right_inotocin_diffeo_pairwise_means)

# Create a DataFrame to combine both sets of data
df_inotocin_affine_left = pd.DataFrame({
    "pairwise_mean": left_inotocin_affine_pairwise_means,
    "set": "Left"
})

df_inotocin_affine_right = pd.DataFrame({
    "pairwise_mean": right_inotocin_affine_pairwise_means,
    "set": "Right"
})

df_inotocin_affine = pd.concat([df_inotocin_affine_left, df_inotocin_affine_right], ignore_index=True)

df_inotocin_diffeo_left = pd.DataFrame({
    "pairwise_mean": left_inotocin_diffeo_pairwise_means,
    "set": "Left"
})

df_inotocin_diffeo_right = pd.DataFrame({
    "pairwise_mean": right_inotocin_diffeo_pairwise_means,
    "set": "Right"
})

df_inotocin_diffeo = pd.concat([df_inotocin_diffeo_left, df_inotocin_diffeo_right], ignore_index=True)

/Users/lindsey/Documents/GitHub/FrankandLopes2025-RefBrain/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lindsey/Documents/GitHub/FrankandLopes2025-RefBrain/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/lindsey/Documents/GitHub/FrankandLopes2025-RefBrain/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/lindsey/Documents/GitHub/FrankandLopes2025-RefBrain/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/lindsey/Documents/GitHub/FrankandLopes2025-RefBrain/.venv/lib/python3.9/site-packages/numpy/core/_methods